In [15]:
import os

import numpy as np

import tensorflow as tf
assert tf.__version__.startswith('2')

from tflite_model_maker import model_spec
from tflite_model_maker import image_classifier
from tflite_model_maker.config import ExportFormat
from tflite_model_maker.config import QuantizationConfig
from tflite_model_maker.image_classifier import DataLoader

import matplotlib.pyplot as plt

import cv2

In [7]:
data = DataLoader.from_folder(r"./datasets/tensorflow/photos/")
train_data, test_data = data.split(0.9)

INFO:tensorflow:Load image with size: 2917, num_label: 2, labels: Moving, still.


INFO:tensorflow:Load image with size: 2917, num_label: 2, labels: Moving, still.


In [8]:
model = image_classifier.create(train_data)

INFO:tensorflow:Retraining the models...


INFO:tensorflow:Retraining the models...


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 hub_keras_layer_v1v2_1 (Hub  (None, 1280)             3413024   
 KerasLayerV1V2)                                                 
                                                                 
 dropout_1 (Dropout)         (None, 1280)              0         
                                                                 
 dense_1 (Dense)             (None, 2)                 2562      
                                                                 
Total params: 3,415,586
Trainable params: 2,562
Non-trainable params: 3,413,024
_________________________________________________________________
None
Epoch 1/5
82/82 [==============================] - 3s 29ms/step - loss: 0.4151 - accuracy: 0.8624
Epoch 2/5
82/82 [==============================] - 2s 29ms/step - loss: 0.3044 - accuracy: 0.9428
Epoch 3/5
82/82 [====================

In [9]:
loss, accuracy = model.evaluate(test_data)

10/10 [==============================] - 1s 28ms/step - loss: 0.2613 - accuracy: 0.9726


In [21]:
video_path = "./datasets/vid2.mp4"
output_path = "resized_video.mp4"
target_width = 224
target_height = 224

resize_video(video_path, output_path, target_width, target_height)

# Load the video using OpenCV
cap = cv2.VideoCapture(output_path)

# Initialize an empty list to store the frame classifications
frame_classifications = []

# Loop through the frames of the video
while True:
    # Read the next frame
    ret, frame = cap.read()

    # Break the loop if the video has ended
    if not ret:
        break

    # Convert the frame to RGB format
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    print(f"shape = {frame_rgb.shape}")
    
    # Classify the frame
    interpreter = tf.lite.Interpreter(model_path="model.tflite")
    interpreter.allocate_tensors()
    output = interpreter.get_output_details()  # Get all output details

    # Model has single output, so access the first element
    output_index = output[0]['index']

    input_details = interpreter.get_input_details()[0]  # Model has single input.
    frame_rgb = np.expand_dims(frame_rgb, axis=0)
    input_data = frame_rgb.astype(np.float32)  # Ensure input data type matches the model

    interpreter.set_tensor(input_details['index'], input_data)
    interpreter.invoke()

    output_data = interpreter.get_tensor(output[0]['index'])
    print(output_data)
'''
    # Add the frame classification to the list
    frame_classifications.append(label)

    # Display the frame
    cv2.imshow("Video", frame)

    # Break the loop if the 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        '''

# Release the video capture and close the OpenCV windows
cap.release()
#cv2.destroyAllWindows()

# Convert the frame classifications to a numpy array
frame_classifications = np.array(frame_classifications)

# Print the frame classifications


print("Frame Classifications:", frame_classifications)

Video resized successfully!
shape = (224, 224, 3)


KeyError: 0

In [11]:
model.export(export_dir='.')

2023-05-26 09:14:16.251543: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: /tmp/tmp4umafjyf/assets


INFO:tensorflow:Assets written to: /tmp/tmp4umafjyf/assets
2023-05-26 09:14:19.503374: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-26 09:14:19.503486: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 1
2023-05-26 09:14:19.503538: I tensorflow/core/grappler/clusters/single_machine.cc:358] Starting new session
2023-05-26 09:14:19.503827: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-26 09:14:19.503946: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-26 09:14:19.504087: I t

INFO:tensorflow:Label file is inside the TFLite model with metadata.


fully_quantize: 0, inference_type: 6, input_inference_type: 3, output_inference_type: 3
INFO:tensorflow:Label file is inside the TFLite model with metadata.


INFO:tensorflow:Saving labels in /tmp/tmp2uz_hpi3/labels.txt


INFO:tensorflow:Saving labels in /tmp/tmp2uz_hpi3/labels.txt


INFO:tensorflow:TensorFlow Lite model exported successfully: ./model.tflite


INFO:tensorflow:TensorFlow Lite model exported successfully: ./model.tflite
